In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist

# 加载 MNIST 数据集
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 创建保存路径
import os
save_path = 'test_images'
os.makedirs(save_path, exist_ok=True)

# 保存一些图片为 PNG
def save_images(images, labels, indices):
    for i in indices:
        img = images[i]
        label = labels[i]
        file_name = f"{save_path}/{label}_{i}.png"
        plt.imsave(file_name, img, cmap='gray')
        print(f"保存图片: {file_name}")

# 从测试集里保存 3 张图片 (你可以指定其他索引)
save_images(x_test, y_test, [0, 3, 9])  # 保存索引 0, 1, 3 处的图片

保存图片: test_images/7_0.png
保存图片: test_images/0_3.png
保存图片: test_images/9_9.png


In [2]:
import os
import tensorflow as tf
from tensorflow.keras.datasets.mnist import load_data
# 忽略警告
import warnings
warnings.filterwarnings('ignore', category=UserWarning)


class DataSource:
    def __init__(self):
        # 数据集路径配置（如果需要本地路径）
#         data_path = os.path.abspath(os.path.dirname(__file__)) + '/data/mnist.npz'
#         (x_train, y_train), (x_test, y_test) = load_data(path=data_path)

        # 数据集路径配置（使用当前工作目录）
        data_path = os.path.join(os.getcwd(), 'data', 'mnist.npz')

        # 加载数据集（如果本地有文件则加载本地，否则从网上下载）
        if os.path.exists(data_path):
            (x_train, y_train), (x_test, y_test) = load_data(path=data_path)
        else:
            print("mnist.npz 文件不存在，从网上下载数据集...")
            (x_train, y_train), (x_test, y_test) = load_data()

        # 为数据增加一个通道维度
        x_train = x_train[..., tf.newaxis]
        x_test = x_test[..., tf.newaxis]

        # 将像素值缩放到 0 ~ 1 范围内
        x_train, x_test = x_train / 255.0, x_test / 255.0

        # 将数据存储为类的属性
        self.train_images, self.train_labels = x_train, y_train
        self.test_images, self.test_labels = x_test, y_test

# 创建 DataSource 实例，加载数据
data = DataSource()

# 打印训练集和测试集的形状
print(f"训练集图片形状: {data.train_images.shape}")
print(f"训练集标签形状: {data.train_labels.shape}")
print(f"测试集图片形状: {data.test_images.shape}")
print(f"测试集标签形状: {data.test_labels.shape}")

# 验证是否为单通道
print(f"训练集单张图片通道数: {data.train_images.shape[-1]}")
print(f"测试集单张图片通道数: {data.test_images.shape[-1]}")



训练集图片形状: (60000, 28, 28, 1)
训练集标签形状: (60000,)
测试集图片形状: (10000, 28, 28, 1)
测试集标签形状: (10000,)
训练集单张图片通道数: 1
测试集单张图片通道数: 1


In [3]:
# 步骤2：搭建卷积神经网络（CNN模型）
import tensorflow as tf
from tensorflow.keras import layers, models

class CNN:
    def __init__(self):
        model = models.Sequential()
        
        # 第1层卷积层 + 最大池化层
        model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
        model.add(layers.MaxPooling2D((2, 2)))

        # 第2层卷积层 + 最大池化层
        model.add(layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))

        # 第3层卷积层
        model.add(layers.Conv2D(64, (3, 3), activation='relu'))

        # 展平层 + 全连接层
        model.add(layers.Flatten())
        model.add(layers.Dense(64, activation='relu'))
        
        # 输出层（10类，使用softmax激活函数）
        model.add(layers.Dense(10, activation='softmax'))

        # 打印模型结构
        model.summary()

        self.model = model



In [4]:
# 步骤2：搭建卷积神经网络（CNN模型）
class Train:
    def __init__(self):
        self.network = CNN()  # 创建CNN模型实例
        self.data = DataSource()  # 加载数据集

    def train(self):
        # 设置模型检查点路径（每5个epoch保存一次）
        check_path = './ckpt/cp-{epoch:04d}.ckpt'
        save_model_cb = tf.keras.callbacks.ModelCheckpoint(
            check_path, save_weights_only=True, verbose=1, save_freq='epoch'
        )

        # 编译模型
        self.network.model.compile(optimizer='adam',
                                   loss='sparse_categorical_crossentropy',
                                   metrics=['accuracy'])

        # 训练模型
        self.network.model.fit(self.data.train_images, 
                               self.data.train_labels, 
                               epochs=10, 
                               callbacks=[save_model_cb])

        # 测试模型
        test_loss, test_acc = self.network.model.evaluate(self.data.test_images, 
                                                          self.data.test_labels)
        print(f"准确率：{test_acc * 100:.2f}%，共测试了 {len(self.data.test_labels)} 张图片")


In [5]:
# 步骤3：训练模型与保存结果
import tensorflow as tf
from tensorflow.keras import layers, models

class CNN:
    def __init__(self):
        model = models.Sequential()
        
        # 第1层卷积层 + 最大池化层
        model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
        model.add(layers.MaxPooling2D((2, 2)))

        # 第2层卷积层 + 最大池化层
        model.add(layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))

        # 第3层卷积层
        model.add(layers.Conv2D(64, (3, 3), activation='relu'))

        # 展平层 + 全连接层
        model.add(layers.Flatten())
        model.add(layers.Dense(64, activation='relu'))
        
        # 输出层（10类，使用softmax激活函数）
        model.add(layers.Dense(10, activation='softmax'))

        # 打印模型结构
        model.summary()

        self.model = model


In [ ]:

# 步骤4：可视化训练过程（TensorBoard）
import numpy as np
from datetime import datetime
from tensorflow.keras.callbacks import TensorBoard

class Train:
    def __init__(self):
        self.network = CNN()  # 创建CNN实例
        self.data = DataSource()  # 加载数据集

    def train(self):
        # 配置TensorBoard日志路径
        logdir = "./logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_cb = TensorBoard(log_dir=logdir)

        # 编译模型
        self.network.model.compile(optimizer='adam',
                                   loss='sparse_categorical_crossentropy',
                                   metrics=['accuracy'])

        # 训练模型并进行可视化监控
        training_history = self.network.model.fit(self.data.train_images,
                                                  self.data.train_labels,
                                                  epochs=10,
                                                  validation_data=(self.data.test_images,
                                                                   self.data.test_labels),
                                                  callbacks=[tensorboard_cb])

        # 打印最终测试结果
        test_loss, test_acc = self.network.model.evaluate(self.data.test_images, 
                                                          self.data.test_labels)
        print(f"准确率：{test_acc * 100:.2f}%，共测试了 {len(self.data.test_labels)} 张图片")
        print("平均误差：", np.average(training_history.history['loss']))

if __name__ == "__main__":
    mnist_train = Train()
    mnist_train.train()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 576)               0

In [ ]:
# 步骤5：预测新图片
from PIL import Image
import numpy as np
# 忽略警告
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

class Predict:
    def __init__(self):
        self.network = CNN()  # 创建CNN实例
        # 加载训练好的模型权重
        #self.network.model.load_weights('./ckpt/cp-0004.ckpt')
        # 在训练结束后保存模型为 .h5 文件
        self.model = tf.keras.models.load_model('./ckpt/model.h5')


    def predict(self, image_path):
        # 读取并处理图片
        img = Image.open(image_path).convert('L')
        flatten_img = np.reshape(img, (28, 28, 1))
        x = np.array([1 - flatten_img])

        # 进行预测
        y = self.network.model.predict(x)
        print(f"{image_path} -> 预测数字为：{np.argmax(y[0])}")

# 测试预测
if __name__ == "__main__":
    app = Predict()
    app.predict('./test_images/7_0.png')
    app.predict('./test_images/9_9.png')
    app.predict('./test_images/0_3.png')